<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/vk_group_all_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
token = 'b20e2fa2b20e2fa2b20e2fa27cb12e31c3bb20eb20e2fa2d50b267b955d1702e85d5db5'
version = '5.131'
domain = 'pravkarelia'

In [ ]:
import pandas as pd
import requests

In [ ]:
def get_all_posts(token, version, domain):
  offset=0
  all_posts = []
  while True:
    responce = requests.get('https://api.vk.com/method/wall.get',
                        params={
                            'access_token' : token,
                            'v': version,
                            'domain': domain,
                            'offset': offset,
                            'count': 100
                        })
    data = responce.json()
    if 'response' in data:
        posts = data['response']['items']
        all_posts.extend(posts)
        if len(posts) < 100:
            return all_posts
        offset += 100
    else:
        return


In [ ]:
posts = get_all_posts(token, version, domain)


In [ ]:
df = pd.DataFrame(posts)

In [ ]:
df.shape

(5644, 22)

In [ ]:
df.head(10)

,inner_type,copy_history,donut,comments,marked_as_ads,hash,type,attachments,date,from_id,...,owner_id,post_type,reposts,text,views,carousel_offset,edited,zoom_text,marked_as_author_ad,copyright
0,wall_wallpost,"[{'inner_type': 'wall_wallpost', 'type': 'post...",{'is_donut': False},{'count': 0},0,M9pot7FTjDjULT_-bg,post,[],1728562921,-187624263,...,-187624263,post,{'count': 0},,{'count': 141},NaN,NaN,NaN,NaN,NaN
1,wall_wallpost,"[{'inner_type': 'wall_wallpost', 'type': 'post...",{'is_donut': False},{'count': 0},0,ic8jJw-h32bG1Fk8nQ,post,[],1728562872,-187624263,...,-187624263,post,{'count': 0},,{'count': 80},NaN,NaN,NaN,NaN,NaN
2,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,Hp3rD5XPbPcDwNB3FQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728498200,-187624263,...,-187624263,post,{'count': 4},📸 Хиротесия во чтецы накануне престольного пра...,{'count': 957},NaN,NaN,NaN,NaN,NaN
3,wall_wallpost,NaN,{'is_donut': False},{'count': 1},0,aolWd_YYkNWN3l0osQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728497802,-187624263,...,-187624263,post,{'count': 1},​10 октября Память преподобного Савватия Солов...,{'count': 639},NaN,NaN,NaN,NaN,NaN
4,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,ovxSRQVGdDsgJUwVVQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728478707,-187624263,...,-187624263,post,{'count': 2},📖 Интересный факт: \n\n🔸 - В источниках V века...,{'count': 463},NaN,NaN,NaN,NaN,NaN
5,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,ZxCDOxNcen7E6k_ZIA,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728464323,-187624263,...,-187624263,post,{'count': 2},МИТРОПОЛИТ КОНСТАНТИН ПОЗДРАВИЛ ИГУМЕНА ИОАННА...,{'count': 315},0.0,NaN,NaN,NaN,NaN
6,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,6bE7JEpmP7Tt8WKlPQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728464218,-187624263,...,-187624263,post,{'count': 0},МИТРОПОЛИТ КОНСТАНТИН: «СВЯТАЯ ЦЕРКОВЬ ДАЕТ НА...,{'count': 119},0.0,1.728464e+09,NaN,NaN,NaN
7,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,OT-fGlzTuqSdA2zGiQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728451560,-187624263,...,-187624263,post,{'count': 2},Из апостольского чтения сегодня: \n\n«И мы поз...,{'count': 339},NaN,NaN,NaN,NaN,NaN
8,wall_wallpost,"[{'inner_type': 'wall_wallpost', 'type': 'post...",{'is_donut': False},{'count': 0},0,WoSB-WucEzIUFnGrsA,post,[],1728415847,-187624263,...,-187624263,post,{'count': 0},,{'count': 436},NaN,NaN,NaN,NaN,NaN
9,wall_wallpost,NaN,{'is_donut': False},{'count': 0},0,9CRZHhF6DDuVS6TemQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1728402115,-187624263,...,-187624263,post,{'count': 0},Завтра память преставления любимого ученика Сп...,{'count': 273},NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.drop(['inner_type', 'donut', 'copy_history', 'hash', 'type', 'attachments'], axis=1)

KeyError: "['inner_type', 'donut', 'copy_history', 'hash', 'type', 'attachments'] not found in axis"

In [ ]:
df.head(5)

,comments,marked_as_ads,date,from_id,id,likes,owner_id,post_type,reposts,text,views,carousel_offset,edited,zoom_text,marked_as_author_ad,copyright
0,{'count': 0},0,1728562921,-187624263,6747,"{'can_like': 0, 'count': 8, 'user_likes': 0}",-187624263,post,{'count': 0},,{'count': 141},NaN,NaN,NaN,NaN,NaN
1,{'count': 0},0,1728562872,-187624263,6746,"{'can_like': 0, 'count': 3, 'user_likes': 0}",-187624263,post,{'count': 0},,{'count': 80},NaN,NaN,NaN,NaN,NaN
2,{'count': 0},0,1728498200,-187624263,6744,"{'can_like': 0, 'count': 47, 'user_likes': 0}",-187624263,post,{'count': 4},📸 Хиротесия во чтецы накануне престольного пра...,{'count': 957},NaN,NaN,NaN,NaN,NaN
3,{'count': 1},0,1728497802,-187624263,6743,"{'can_like': 0, 'count': 42, 'user_likes': 0}",-187624263,post,{'count': 1},​10 октября Память преподобного Савватия Солов...,{'count': 639},NaN,NaN,NaN,NaN,NaN
4,{'count': 0},0,1728478707,-187624263,6742,"{'can_like': 0, 'count': 12, 'user_likes': 0}",-187624263,post,{'count': 2},📖 Интересный факт: \n\n🔸 - В источниках V века...,{'count': 463},NaN,NaN,NaN,NaN,NaN


In [ ]:
new_df = df[['date', 'id', 'likes', 'reposts', 'text', 'views']]


In [ ]:
new_df ['date'] = pd.to_datetime(df['date'], unit='s')

<ipython-input-36-8ded89f5aa1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df ['date'] = pd.to_datetime(df['date'], unit='s')


In [ ]:
new_df.head()

,date,id,likes,reposts,text,views
0,2024-10-10 12:22:01,6747,"{'can_like': 0, 'count': 8, 'user_likes': 0}",{'count': 0},,{'count': 141}
1,2024-10-10 12:21:12,6746,"{'can_like': 0, 'count': 3, 'user_likes': 0}",{'count': 0},,{'count': 80}
2,2024-10-09 18:23:20,6744,"{'can_like': 0, 'count': 47, 'user_likes': 0}",{'count': 4},📸 Хиротесия во чтецы накануне престольного пра...,{'count': 957}
3,2024-10-09 18:16:42,6743,"{'can_like': 0, 'count': 42, 'user_likes': 0}",{'count': 1},​10 октября Память преподобного Савватия Солов...,{'count': 639}
4,2024-10-09 12:58:27,6742,"{'can_like': 0, 'count': 12, 'user_likes': 0}",{'count': 2},📖 Интересный факт: \n\n🔸 - В источниках V века...,{'count': 463}


In [ ]:
new_df.to_csv('pravkarelia_all_posts.csv', index=False)

In [ ]:
new_df.head()

,date,id,likes,reposts,text,views
0,2024-10-10 12:22:01,6747,"{'can_like': 0, 'count': 8, 'user_likes': 0}",{'count': 0},,{'count': 141}
1,2024-10-10 12:21:12,6746,"{'can_like': 0, 'count': 3, 'user_likes': 0}",{'count': 0},,{'count': 80}
2,2024-10-09 18:23:20,6744,"{'can_like': 0, 'count': 47, 'user_likes': 0}",{'count': 4},📸 Хиротесия во чтецы накануне престольного пра...,{'count': 957}
3,2024-10-09 18:16:42,6743,"{'can_like': 0, 'count': 42, 'user_likes': 0}",{'count': 1},​10 октября Память преподобного Савватия Солов...,{'count': 639}
4,2024-10-09 12:58:27,6742,"{'can_like': 0, 'count': 12, 'user_likes': 0}",{'count': 2},📖 Интересный факт: \n\n🔸 - В источниках V века...,{'count': 463}


In [ ]:
new_df['likes'] = new_df['likes'].apply(lambda x: x['count'])

<ipython-input-42-cf1af49ebc83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['likes'] = new_df['likes'].apply(lambda x: x['count'])


In [ ]:
new_df.head()

,date,id,likes,reposts,text,views
0,2024-10-10 12:22:01,6747,8,{'count': 0},,{'count': 141}
1,2024-10-10 12:21:12,6746,3,{'count': 0},,{'count': 80}
2,2024-10-09 18:23:20,6744,47,{'count': 4},📸 Хиротесия во чтецы накануне престольного пра...,{'count': 957}
3,2024-10-09 18:16:42,6743,42,{'count': 1},​10 октября Память преподобного Савватия Солов...,{'count': 639}
4,2024-10-09 12:58:27,6742,12,{'count': 2},📖 Интересный факт: \n\n🔸 - В источниках V века...,{'count': 463}


In [ ]:
new_df['reposts'] = new_df['reposts'].apply(lambda x: x['count'])

<ipython-input-45-b37741aa55d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['reposts'] = new_df['reposts'].apply(lambda x: x['count'])


In [ ]:
new_df['views'] = new_df['views'].apply(lambda x: x['count'])

<ipython-input-46-3a202ba5a94b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['views'] = new_df['views'].apply(lambda x: x['count'])


In [ ]:
new_df['text'] = new_df['text'].str.replace('\n', ' ', regex=False)

<ipython-input-47-c07428bbf4e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['text'] = new_df['text'].str.replace('\n', ' ', regex=False)


In [ ]:
new_df.head()

,date,id,likes,reposts,text,views
0,2024-10-10 12:22:01,6747,8,0,,141
1,2024-10-10 12:21:12,6746,3,0,,80
2,2024-10-09 18:23:20,6744,47,4,📸 Хиротесия во чтецы накануне престольного пра...,957
3,2024-10-09 18:16:42,6743,42,1,​10 октября Память преподобного Савватия Солов...,639
4,2024-10-09 12:58:27,6742,12,2,📖 Интересный факт: 🔸 - В источниках V века н...,463


In [ ]:
new_df['date'] = new_df['date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow')

<ipython-input-49-5edbb890c301>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['date'] = new_df['date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Moscow')


In [ ]:
result_df = new_df[['id','text', 'date', 'likes','reposts','views']]

In [ ]:
result_df.head(5)

,id,text,date,likes,reposts,views
0,6747,,2024-10-10 15:22:01,8,0,141
1,6746,,2024-10-10 15:21:12,3,0,80
2,6744,📸 Хиротесия во чтецы накануне престольного пра...,2024-10-09 21:23:20,47,4,957
3,6743,​10 октября Память преподобного Савватия Солов...,2024-10-09 21:16:42,42,1,639
4,6742,📖 Интересный факт: 🔸 - В источниках V века н...,2024-10-09 15:58:27,12,2,463


In [ ]:
# Удаление информации о временной зоне и преобразование в строку
result_df['date'] = result_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
result_df.to_csv('pravkarelia_all_posts.csv', index=False)